#### RULE-1: Header to be used as metadata
#### RULE-2: Convert column names into lowercase
#### RULE-3: If a given record is missing visitor (just the last name of visitor) or visitee (just the last name of visitee) , 
#### then that record is dropped from all calculations
#### RULE-4: All input must be converted to lowercase letters
#### RULE-5: Your solution has to be generic and should be able to handle billions of records
#### RULE-6: You have to pass input as a parameter to your PySpark program

In [2]:
#importing findspark
import findspark
findspark.init()

In [3]:
# create a SparkSession object
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import count, col, min, max
spark = SparkSession.builder.master("local").appName("whitehouselog").getOrCreate()
#spark.conf.set('spark.sql.caseSensitive', True)
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

In [3]:
#To read the csv data together with header
df_withheader = spark.read.csv("/Users/C940/spark3/whitehouselog.csv")

In [4]:
#To get the total count of data plus the header
df_withheader.count()

970505

In [5]:
# Rule 1: To make the header as the metadata
df = spark.read.csv("/Users/C940/spark3/whitehouselog.csv", header=True)

In [6]:
#Print schema of dataframe
df.printSchema()

root
 |-- NAMELAST: string (nullable = true)
 |-- NAMEFIRST: string (nullable = true)
 |-- NAMEMID: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- BDGNBR: string (nullable = true)
 |-- ACCESS_TYPE: string (nullable = true)
 |-- TOA: string (nullable = true)
 |-- POA: string (nullable = true)
 |-- TOD: string (nullable = true)
 |-- POD: string (nullable = true)
 |-- APPT_MADE_DATE: string (nullable = true)
 |-- APPT_START_DATE: string (nullable = true)
 |-- APPT_END_DATE: string (nullable = true)
 |-- APPT_CANCEL_DATE: string (nullable = true)
 |-- Total_People: string (nullable = true)
 |-- LAST_UPDATEDBY: string (nullable = true)
 |-- POST: string (nullable = true)
 |-- LASTENTRYDATE: string (nullable = true)
 |-- TERMINAL_SUFFIX: string (nullable = true)
 |-- visitee_namelast: string (nullable = true)
 |-- visitee_namefirst: string (nullable = true)
 |-- MEETING_LOC: string (nullable = true)
 |-- MEETING_ROOM: string (nullable = true)
 |-- CALLER_NAME_LAST: string (

In [7]:
#count of the content excluding header
df.count()

970504

In [8]:
#To show top 20 rows of the dataframe
df.select("NAMELAST","NAMEFIRST", "visitee_namelast", "visitee_namefirst").show()

+---------------+---------+----------------+-----------------+
|       NAMELAST|NAMEFIRST|visitee_namelast|visitee_namefirst|
+---------------+---------+----------------+-----------------+
|TAJOURIBESSASSI|   HANENE|        Pelofsky|             Eric|
|        bageant|    laura|     Baskerville|           Steven|
|       Broemson|     Earl|     Baskerville|           Steven|
|    Jackling Jr|  William|     Baskerville|           Steven|
|        McCrary|  Richard|     Baskerville|           Steven|
|        Mulcahy|   Joshua|     Baskerville|           Steven|
|           Ryan|   Oliver|     Baskerville|           Steven|
|       Smith Jr|  William|     Baskerville|           Steven|
|         Keeler|  Douglas|       Goldstein|             Jeff|
|          DAVIS|   JUSTIN|            Drew|              MAJ|
|         Glover|   Vinson|         Lengyel|            jason|
|         Ambler|   Andrew|          OFFICE|         VISITORS|
|         Ambler|     John|          OFFICE|         VI

In [ ]:
#Rule 2- To convert column names into lowercase
df2 = df.select([f.col(x).alias(x.lower()) for x in df.columns])

In [10]:
def lower_case(str):
    return str.lower()


In [11]:
df.withColumn(”Lowercasename", lower_case(col("NAM"))).show(truncate=False)

SyntaxError: invalid character '”' (U+201D) (Temp/ipykernel_53336/2036632366.py, line 1)

In [ ]:
df2.printSchema()

In [ ]:
#Rule 3-If a given record is missing visitor (just the last name of visitor), dropped from calculation
df3 = df2.filter(df2.namelast.isNotNull())

In [ ]:
#Count of df3
df3.count()

In [ ]:
#Rule 3-If a given record is missing visitee (just the last name of visitee), dropped from calculation
df_final = df3.filter(df3.visitee_namelast.isNotNull())

In [ ]:
#Final count after record for missing visitee (last name) and visitor(last name) dropped from calculation
df_final.count() 

In [ ]:
#To view top 20 rows
df_final.select("namelast","namefirst", "visitee_namelast", "visitee_namefirst").show()

In [ ]:
#Rule 4- All input to be converted to lowercase letters
expression = [ f.lower(f.col(x)).alias(x) for x in df_final.columns ]

In [ ]:
df_lower = df_final.select(*expression)

In [ ]:
#To check if all input is converted into lowercase letters
df_lower.select('namelast', 'namefirst', 'visitee_namelast', 'visitee_namefirst').show()

## a) To retrieve Top 10 frequent visitors to whitehouse

In [ ]:
#To concatenate the visitor namelast and namefirst into fullname_visitor
df_visitor = df_lower.withColumn('fullname_visitor', f.concat(f.col('namelast'), f.lit(','), f.col('namefirst')))

In [ ]:
#To show the full name (visitor) of namelast and namefirst without truncation
df_visitor.select('fullname_visitor').show(truncate=False)

In [ ]:
#To group by fullname_visitor and get the count/frequency of names appearing in the dataframe
df_visitor_findtop10 = df_visitor.groupBy('fullname_visitor').agg(count('fullname_visitor').alias('frequency'))

In [ ]:
#To orderby count/frequency in descending order
df_top10_visitor = df_visitor_findtop10.orderBy(col('frequency').desc())

In [ ]:
#To display top 10 visitors to whitehouse in a dataframe
df_top10_visitor.limit(10).show()

## b) To retrieve Top 10 frequent visitee to whitehouse

In [ ]:
#To concatenate the visitee namelast and vsitee namefirst into fullname_visitee
df_visitee=df_lower.withColumn('fullname_visitee',f.concat(f.col('visitee_namelast'),f.lit(','),f.col('visitee_namefirst')))

In [ ]:
#To show the full name of visitee namelast and namefirst without truncation
df_visitee.select('fullname_visitee').show()

In [ ]:
#To group by fullname_visitee and get the count/frequency of names appearing in the dataframe
df_visitee_findtop10 = df_visitee.groupBy('fullname_visitee').agg(count('fullname_visitee').alias('frequency'))

In [ ]:
#To orderby count/frequency in descending order
df_top10_visitee = df_visitee_findtop10.orderBy(col('frequency').desc())

In [ ]:
#To display top 10 visitee to whitehouse in a dataframe
df_top10_visitee.limit(10).show()

## c) To retrieve Top 10 frequent visitor-visitee to whitehouse

In [ ]:
#To concatenate the visitor-visitee namelast and namefirst into fullname_visitorvisitee
df_visitor_visitee = df_lower.withColumn("fullname_visitorvisitee",f.concat(col("namelast"),f.lit(','),
    col("namefirst"),f.lit(' - '),col("visitee_namelast"), f.lit(','), col("visitee_namefirst")))

In [ ]:
#To show the fullname_visitorvisitee without truncation
df_visitor_visitee.select("fullname_visitorvisitee").show(truncate=False)

In [ ]:
#To group by fullname_visitorvisitee and get the count/frequency of names appearing in the dataframe
df_visitorvisitee_findtop10=df_visitor_visitee.groupBy('fullname_visitorvisitee').\
                           agg(count('fullname_visitorvisitee').alias('frequency'))

In [ ]:
#To orderby count/frequency in descending order
df_top10_visitor_visitee = df_visitorvisitee_findtop10.orderBy(col('frequency').desc())

In [ ]:
#To display top 10 visitor-visitee to whitehouse in a dataframe
df_top10_visitor_visitee.limit(10).show(truncate=False)

## d) Number of records dropped from the data

In [ ]:
#Total number of records including header minus Final count after record for missing visitee & visitor (last name)
difference = df_withheader.count() - df_final.count()
difference
print("The number of records dropped from data is:", difference)

## To stop spark session